In [4]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [5]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import (
    read_indexer_communities,
    read_indexer_entities,
    read_indexer_reports,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [ ]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.encoding_for_model(llm_model)

### Load community reports as context for global search

- Load all community reports in the `community_reports` table from GraphRAG, to be used as context data for global search.
- Load entities from the `entities` tables from GraphRAG, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the rank attribute in the community reports table for context ranking)
- Load all communities in the `communities` table from the GraphRAG, to be used to reconstruct the community graph hierarchy for dynamic community selection.

In [7]:
# parquet files generated from indexing pipeline
INPUT_DIR = "./inputs/operation dulce"
COMMUNITY_TABLE = "communities"
COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [8]:
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

communities = read_indexer_communities(community_df, report_df)
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)

report_df.head()

Total report count: 72
Report count after filtering by community level 2: 56


,id,human_readable_id,community,parent,level,title,summary,full_content,rank,rank_explanation,findings,full_content_json,period,size
0,16949a5d17b740b2b4a6f787b0a637f1,43,43,10,2,Ben Bloomberg and the Harmoniser Project,"The community centers around Ben Bloomberg, a ...",# Ben Bloomberg and the Harmoniser Project\n\n...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'Ben Bloomberg is a pivotal f...,"{\n ""title"": ""Ben Bloomberg and the Harmoni...",2025-01-10,35
1,4ff756b7041f4dcab6612e016af2b14d,44,44,10,2,North Hampton and Influential Musicians,"The community centers around North Hampton, a ...",# North Hampton and Influential Musicians\n\nT...,6.5,The impact severity rating is moderately high ...,[{'explanation': 'North Hampton serves as the ...,"{\n ""title"": ""North Hampton and Influential...",2025-01-10,4
2,2d3df394272743a781606ad80ccb5312,45,45,10,2,Prince of Monaco and Monaco,The community revolves around the Prince of Mo...,# Prince of Monaco and Monaco\n\nThe community...,4.0,The impact severity rating is moderate due to ...,[{'explanation': 'The Prince of Monaco is a ke...,"{\n ""title"": ""Prince of Monaco and Monaco"",...",2025-01-10,2
3,becbd958973f42b0bd53cca9250feaf1,46,46,10,2,Robot Opera and Broadway,"The community revolves around the Robot Opera,...",# Robot Opera and Broadway\n\nThe community re...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'The Robot Opera is a notable...,"{\n ""title"": ""Robot Opera and Broadway"",\n ...",2025-01-10,2
4,f7d29921ae3e41a79ae7f88dae584892,47,47,13,2,Ben and Jacob's Fusion of Art and Technology,"The community centers around Ben and Jacob, wh...",# Ben and Jacob's Fusion of Art and Technology...,7.5,The impact severity rating is high due to the ...,[{'explanation': 'Ben and Jacob are key collab...,"{\n ""title"": ""Ben and Jacob's Fusion of Art...",2025-01-10,5


#### Build global context based on community reports

In [9]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    communities=communities,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [10]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [11]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [12]:
result = await search_engine.asearch(
    "What is Cosmic Vocalization and who are involved in it?"
)

print(result.response)

### Cosmic Vocalization: An Overview

Cosmic Vocalization is a term coined by Jordan Hayes to describe a repeating sequence found in cryptic communications. This concept is pivotal in the realm of interstellar communication, serving as a common reference point for both humanity and extraterrestrial entities. The repeating sequence identified by Hayes is crucial for understanding and interpreting the signals exchanged during what is known as the Interstellar Duet [Data: Reports (65)].

### Key Figures and Entities

#### Jordan Hayes
Jordan Hayes is a central figure in the development and understanding of Cosmic Vocalization. Hayes' work in identifying and describing the repeating sequence in cryptic communications has been instrumental in the ongoing efforts to interact with extraterrestrial intelligence. His contributions provide the foundational knowledge required to decode and engage with these alien signals [Data: Reports (65)].

#### Paranormal Military Squad
The Paranormal Militar

In [13]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,50,Alex Mercer and the Dulce Base Team,0.956522,# Alex Mercer and the Dulce Base Team\n\nThe c...,8.5
1,35,Kevin Scott and Technology Development,0.608696,# Kevin Scott and Technology Development\n\nTh...,7.5
2,53,Dulce Base and Paranormal Military Squad,0.565217,# Dulce Base and Paranormal Military Squad\n\n...,8.5
3,22,Paranormal Military Squad and Technological Ex...,0.434783,# Paranormal Military Squad and Technological ...,8.5
4,60,First Contact with Extraterrestrial Civilization,0.304348,# First Contact with Extraterrestrial Civiliza...,9.5
5,61,Dulce Base Operations and Distress,0.173913,# Dulce Base Operations and Distress\n\nThe co...,8.5
6,57,Operation: Dulce in New Mexico,0.130435,# Operation: Dulce in New Mexico\n\nThe commun...,7.5
7,48,Jacob Collier and Ben Bloomberg's First Tour,0.130435,# Jacob Collier and Ben Bloomberg's First Tour...,7.5
8,51,Cosmic Translators and Alien Script,0.086957,# Cosmic Translators and Alien Script\n\nThe c...,8.5
9,64,Terminal and Deep Hum,0.086957,# Terminal and Deep Hum\n\nThe community revol...,8.5


In [14]:
# inspect number of LLM calls and tokens
print(
    f"LLM calls: {result.llm_calls}. Prompt tokens: {result.prompt_tokens}. Output tokens: {result.output_tokens}."
)

LLM calls: 4. Prompt tokens: 33015. Output tokens: 655.
